In [1]:
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [14]:
from typing import List, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.chat_models import init_chat_model

# Initialise LLM (Gemini 2.5)
llm = init_chat_model(
    model="gemini-2.5-flash",
    model_provider="google_genai",
    temperature=0.0
)

# Canonical user input
paragraph = (
    "I identified these five genes to be significantly more mutated than expected by chance in my cohort of human brain cancer patients: TP53, AKT3, EGFR, ATRX and PDX1. Do a GSEA"
)

# Define Pydantic class for input genes and context
class StudyExtraction(BaseModel):
    genes: List[str] = Field(
        description="List of gene symbols mentioned in the text, normalized to official HGNC/NCBI-style symbols if possible."
    )
    organism: Optional[str] = Field(
        description="Scientific name (binomial) of the organism (e.g., 'Homo sapiens', 'Mus musculus')."
    )
    field_of_study: Optional[str] = Field(
        description="High-level biomedical domain, e.g., 'oncology', 'cancer genomics', 'neuroscience', 'immunology', 'microbiology'."
    )
    organ: Optional[str] = Field(
        description="Primary organ or tissue referenced (e.g., 'brain', 'liver', 'blood')."
    )
    analysis_type: Optional[str] = Field(
        description="Concise description of the analysis performed, e.g., 'differential expression', 'mutation enrichment', 'GWAS', 'copy-number analysis', 'metagenomic profiling'."
    )
    GSEA: bool = Field(default=False, 
                       description="Whether the user mentions that a GSEA is needed on the gene set. If no mention, keep it False.")

#alternative implementation to parse as pydantic more robustly
parser = PydanticOutputParser(pydantic_object=StudyExtraction)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract per schema:\n{format_instructions}"),
    ("human", "{paragraph}"),
]).partial(format_instructions=format_instructions)
parsing_llm = prompt | llm | parser

# pass raw user input "paragraph"
parsed_input = parsing_llm.invoke({"paragraph": paragraph})
#convert to JSON string
json_output = parsed_input.model_dump_json(indent=2)
print(json_output)

#next step - inject the json to the LLM to determine attributes to fetch from BioMart


{
  "genes": [
    "TP53",
    "AKT3",
    "EGFR",
    "ATRX",
    "PDX1"
  ],
  "organism": "Homo sapiens",
  "field_of_study": "cancer genomics",
  "organ": "brain",
  "analysis_type": "mutation enrichment analysis",
  "GSEA": true
}


In [ ]:
from src.querries_script import group_by_gene_dynamic, fill_with_ncbi, call_querry_biomart
import pandas as pd    

attributes = pd.read_csv("data/attributes.csv")["name"].to_list()

output = call_querry_biomart(attributes=attributes[:15],
                            filters={"external_gene_name": parsed_input.genes})

output = group_by_gene_dynamic(output)

output = fill_with_ncbi(output)

print(output)

In [20]:
from typing import List
from gseapy import enrichr

def enrichr_query(gene_list: list):
    """Run enrichment analysis on a list of genes.

    This tool allows to run enrichment analysis on a list of genes using the `gseapy` library.
    Using this tool, a model can get information about the biological processes enriched in a set of genes.

    Args:
        gene_list: list of genes to run enrichment analysis on

    Returns:
        DataFrame: DataFrame containing the enrichment results
    """
    # Run enrichment
    enr = enrichr(
        gene_list=gene_list,
        gene_sets='GO_Biological_Process_2021',
        organism='Human',
        outdir=None,  # no files will be written
        cutoff=0.05
    )

    # Save results as DataFrame
    df_results = enr.results

    return df_results

In [43]:
if parsed_input.GSEA: 
    tool_results = enrichr_query(parsed_input.genes)

        # Optionally, format or reduce the output for readability (e.g., top rows)
    if not tool_results.empty:
        # Filter rows where Adjusted P-value < 0.05
        filtered_df = tool_results[tool_results["Adjusted P-value"] < 0.05]

        # Drop the 'Gene_set' column
        filtered_df = filtered_df.drop(columns=["Gene_set","Old P-value","Old Adjusted P-value"])
        gsea_string = "the results of gene set enrichment are:\n "+ filtered_df.head(20).to_string(index = False)
        # Show the top rows
        # print(gsea_string)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

text = str(output)

task ='''
<task>
You are a helpful and biological expert specializing in integrating and interpreting gene-related data from the given information report, combined with the knowledge you have obtained during your training.
When given a report on a set of genes, summarize the key insights for each gene. The final and most important job is to concisely contextualise the user's findings with biological or biomedical background knowledge and find commonalities between the genes given. Stay to-the-point and scientifically accurate. Tailor the response to the context given by the client.
Think hard!
</task>
'''

user_prompt = "I want to know more about these genes {input_g}. They were obtained after {analyses} in {organism}. All information I know about these genes is the following: {text} \n Do you find any commonalities or interesting findings about these genes? I'm mainly interested in {context}"
if parsed_input.GSEA:
    user_prompt += gsea_string
    
prompt = ChatPromptTemplate.from_messages([
    ("system", task),
    ("user", user_prompt)
])
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"text": text, "input_g": parsed_input.genes, "context": parsed_input.field_of_study, "analyses": parsed_input.analysis_type, "organism": parsed_input.organism})
print(response)